In [1]:
from obspy import read, read_events, read_inventory
import os
import sys
from glob import glob
from obspy.signal.trigger import recursive_sta_lta
from obspy.signal.trigger import coincidence_trigger
from obspy.core.event.catalog import Catalog
from obspy.core.event.event import Event
from obspy.core.event.magnitude import StationMagnitude, Magnitude, StationMagnitudeContribution, Amplitude
from obspy.core.event.base import QuantityError, TimeWindow, WaveformStreamID, Comment
from helper_functions import *
%matplotlib inline

In [4]:
cat = read_events("detections/20200310/bhdetections_20200310_*.xml")
catm = Catalog()
for ev in cat:
    if len(ev.magnitudes):
        catm += ev
        
print(catm)
print(UTCDateTime(2020,3,10,0,0,0).strftime("%j"))

34 Event(s) in Catalog:
 | -2.2129028140039715 Mw_spectral
 | -3.2416876991087893 Mw_time
...
 | -2.604842760464418 Mw_time
 | -2.9840448752351674 Mw_spectral
To see all events call 'print(CatalogObject.__str__(print_all=True))'
070


In [15]:
def get_stream_geophone(ev):

    starttime = min([p.time for p in ev.picks]) - 1.0
    endtime = min([p.time for p in ev.picks]) + 2.0
    stations = list(set([p.waveform_id.station_code for p in ev.picks]))
    DATA_DIR_ROOT = "/home/gilbert_lab/cami_frs/borehole_data/sac_daily_nez_500Hz/"
    datestr = starttime.strftime("%Y%m%d")

    stream = read(os.path.join(DATA_DIR_ROOT, datestr, "*.sac"), 
                  starttime=starttime, 
                  endtime=endtime)
    stream.detrend("demean")
    stream.detrend("linear")
    
    return stream

def get_stream_bb(ev):

    starttime = min([p.time for p in ev.picks]) - 1.0
    endtime = min([p.time for p in ev.picks]) + 2.0
    stations = list(set([p.waveform_id.station_code for p in ev.picks]))
    DATA_DIR_ROOT = "/home/gilbert_lab/cami_frs/broadband/data/"
    syr = starttime.strftime("%Y")
    sjul = starttime.strftime("%j")

    stream = read(os.path.join(DATA_DIR_ROOT, syr, sjul, "FRS*", "HH*", "*.MSEED"), 
                  starttime=starttime, 
                  endtime=endtime)
    stream.detrend("demean")
    stream.detrend("linear")
    
    # correct for response
    inv = read_inventory("/home/gilbert_lab/cami_frs/broadband/4E_broadband.xml")
    
    stream.remove_response(inventory=inv)
    
    return stream

In [16]:
def magnitude(station, st, event):

    r_med = distance_from_tstp(event.picks, min_estim=1)
    print("median r = %.2f" % r_med)
    sta_st = st.select(station=station).copy()
    sta_st.detrend()
    #sta_st.plot()

    sta_picks = [p for p in event.picks if p.waveform_id.station_code == station]

    # Estimate coda
    tp = get_pick(event.picks, station, "P")
    ts = get_pick(event.picks, station, "S")
    if not ts:
        return
    if not tp:
        tsig = ts - 0.5
    else:
        tsig = tp - 0.02
    tcoda, s_len, snr = get_coda_duration(sta_st.copy(), tsig=tsig, ts=ts, win_len_s=0.2)

    # Estimate hypocentral distance from ts-tp
    VP = 1800.  # m/s
    VS = 630.  # m/s
    if ts and tp:
        r = (ts - tp) / (1 / VS - 1 / VP)
    else: 
        r = r_med
    print("Hypocentral distance: r = %.2f" % r)

    # Estimate Mw for each component
    Mw_spec_sta = []
    Mw_time_sta = []
    Q_spec_sta = []
    fc_spec_sta = []
    for tr in sta_st:
        print(tr.id)
        
        # Cut noise window and S waveform
        tr0 = tr.copy()
        noise_len = s_len
        taper_perc = 0.1
        trnoise = tr.copy()
        trnoise.trim(starttime=tsig - (1 + taper_perc) * noise_len, endtime=tsig - taper_perc * noise_len)
        trnoise.taper(type="hann", max_percentage=taper_perc, side="both")
        tr.trim(starttime=ts - taper_perc * s_len, endtime=ts + (1 + taper_perc) * s_len)
        tr.taper(type="hann", max_percentage=taper_perc, side="both")
        
        # Check SNR
        snr_trace = np.median(tr.slice(starttime=ts, endtime=ts + s_len).data) / \
                    np.median(trnoise.data)

        if snr_trace < 3:
            Logger.info("SNR < 3, skipping trace for magnitude calculation.")
            # Poor SNR, skip trace
            continue

        # Displacement waveform
        trdisp = tr.copy()
        trdisp.integrate()
        trdisp.detrend()

        # Estimate magnitude: time method
        Mw_time, M0_time, omega0_time = estimate_magnitude_time(trdisp, r, disp=True)
        Mw_time_sta.append(Mw_time)
        print("Mw time estimate: %.2f" % Mw_time)
        
        # plot
        ax = plt.subplot(111)
        tvec0 = tr0.times("matplotlib")
        tvec1 = tr.times("matplotlib")
        ax.plot_date(tvec0, tr0.data, "k")
        ax.plot_date(tvec1, tr.data, "r")
        dateFmt = mdates.DateFormatter('%H:%M:%S')
        ax.xaxis.set_major_formatter(dateFmt)
        plt.xticks(rotation=90)
        plt.title("%s: Mw time estimate = %.2f" % (tr.id, Mw_time))
        plt.show()
        plt.close()
        
        # Estimate magnitude: spectral method
        Mw_o, M0_o, omega0_o, fc_o, Q_o = estimate_magnitude_spectral(trdisp, r, omega0_time, trnoise=None,
                                                                      disp=True)
        if not Mw_o:
            Logger.warning("No magnitude found due to errors.")
            continue
        elif fc_o < 2 or Q_o > 40 or Q_o < 1:  # Qs Attenuation larger than Sandstone=31, shale=10
            # Reject spectral estimate
            Logger.warning("Rejecting spectral estimate with: fc = %f, Q = %f" % (fc_o, Q_o))
            continue
        else:
            Mw_spec_sta.append(Mw_o)
            Q_spec_sta.append(Q_o)
            fc_spec_sta.append(fc_o)
            print("Mw = %.2f, Q = %.2f, Fc = %.1f" % (Mw_o, Q_o, fc_o))
            
    return Mw_spec_sta, Mw_time_sta, Q_spec_sta, fc_spec_sta

In [17]:
nmagn = [len(ev.station_magnitudes) for ev in catm]
catm_sorted = [ev for _, ev in sorted(zip(nmagn, catm), key=lambda x: x[0], reverse=True)]

for ev in catm_sorted:

    print(ev)
    
    #st = get_stream_geophone(ev) 
    st = get_stream_bb(ev) 
    stations = list(set([p.waveform_id.station_code for p in ev.picks]))
    
    for station in stations:
        print("station: ")
        st.select(station=station).plot()
        print("station: %s" % station)
        magnitude(station, st, ev)

Event:	 | -2.374314636492673 Mw_spectral

	        resource_id: ResourceIdentifier(id="smi:local/15c6451b-dd46-4d2a-ae5f-8dc60e7a6988")
	               ---------
	              picks: 20 Elements
	         amplitudes: 20 Elements
	         magnitudes: 2 Elements
	 station_magnitudes: 10 Elements


/home/genevieve.savard/anaconda3/envs/eqcorrscan/lib/python3.6/site-packages/obspy/core/inventory/network.py:254: UserWarning: Found more than one matching response. Returning first.
  warnings.warn(msg)


IndexError: Empty stream object

In [ ]:
# for amp in ev.amplitudes:
#     #print(amp)
#     if amp.type == "A":
#         #print(amp)
#         print("\tSNR = %.2f" % amp.snr)
#         print("\t%s" % amp.waveform_id.station_code)
#         #print("\t%s" % amp.time_window)

# for magn in ev.magnitudes:
    
#     print("Magnitude: \n\t Type: %s\n\t Value= %.2f +/ %.2f" % (magn.magnitude_type, magn.mag, magn.mag_errors.uncertainty))

#     for contrib in magn.station_magnitude_contributions:
#         sid = contrib.station_magnitude_id
#         smag = [m for m in ev.station_magnitudes if m.resource_id == sid][0]
#         print(smag.mag)
#         print(smag.comments[-1].text)
#         #for com in smag.comments:
#         #    print("\t%s" % com.text)